# Classificação de Imagens de Arco senil

In [3]:
from keras.layers import Activation,Convolution2D, Dropout,Conv2D
from keras.layers import AveragePooling2D,BatchNormalization
from keras.layers import GlobalAveragePooling2D
from keras.models import Sequential
from keras.layers import Flatten
from keras.models import Model
from keras.layers import Input
from keras.layers import MaxPooling2D
from keras.layers import SeparableConv2D
from keras import layers
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
import scipy.misc
from imutils import paths
import cv2
import os
import random

In [19]:
def model_cnn_2(classes):
    model = Sequential()
    model.add(Convolution2D(filters=16,kernel_size=(7,7),activation='relu',padding='same',
                           name='image_array',input_shape=(150,150,3)))
    model.add(BatchNormalization())
    model.add(Convolution2D(filters=16,kernel_size=(7,7),activation='relu',
                           strides=(2,2),padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2, 2))
    #model.add(Activation('relu'))
    
    model.add(Convolution2D(filters=48,kernel_size=(5,5),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2, 2))
    model.add(Convolution2D(filters=48,kernel_size=(5,5),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2, 2))
    #model.add(Activation('relu'))
    
    model.add(Convolution2D(filters=128,kernel_size=(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2, 2))
    model.add(Convolution2D(filters=128,kernel_size=(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2, 2))
    #model.add(Activation('relu'))
    
    model.add(Convolution2D(filters=192,kernel_size=(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(Convolution2D(filters=192,kernel_size=(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(Convolution2D(filters=128,kernel_size=(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2, 2))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(4096,activation='relu'))
    model.add(layers.Dense(4096,activation='relu'))
    model.add(layers.Dense(classes,activation='sigmoid'))
    
    return model
    

In [20]:
model = model_cnn_2(2)

In [21]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_array (Conv2D)         (None, 150, 150, 16)      2368      
_________________________________________________________________
batch_normalization_14 (Batc (None, 150, 150, 16)      64        
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 75, 75, 16)        12560     
_________________________________________________________________
batch_normalization_15 (Batc (None, 75, 75, 16)        64        
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 37, 37, 16)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 37, 37, 48)        19248     
_________________________________________________________________
batch_normalization_16 (Batc (None, 37, 37, 48)        192       
__________

In [22]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

carregando imagens 

In [9]:
print("Carregando imagens...")
data = []
labels = []

imagePaths = sorted(list(paths.list_images('/home/jailsonpereira/PAIC_CNN/versao_1')))
random.seed(42)
random.shuffle(imagePaths)

for imagePath in imagePaths:
    image = cv2.imread(imagePath)
    image = cv2.resize(image,(224,224))
    data.append(image)
    
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)

data = np.array(data,dtype='float')/255.0
labels = np.array(labels)

print("Imagens carregadas")

Carregando imagens...
Imagens carregadas


particionando e normalizando dataset

In [10]:
(trainX,testX,trainY,testY) = train_test_split(data,labels,test_size=0.20,random_state=42)

In [11]:
#conveter o labels para inteiros em um vetor (para 2-class, binary)
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

In [13]:
# Data argumentation
aug = ImageDataGenerator(rotation_range=30,width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
    horizontal_flip=True, fill_mode="nearest")